# Hypothesentest

Nachfolgend wird der Hypothesentest nach Spearman durchgeführt. <br>
<br>
Warum Spearman statt Pearson?
- misst monotonen Zusammenhang (nicht nur linear)
- keine Normalverteilung nötig
- robust gegenüber Ausreißern und vielen Nullen
- geeignet für zusammengesetzte / ordinale Variablen

In [ ]:
import pandas as pd
from scipy.stats import spearmanr

konsum = pd.read_csv("../../Daten_sets/wasser_cleaning/alle_staedte2.csv", sep=";")
berichte = pd.read_csv("../../statistik/prozentuale_aenderung_blaulicht_drogen_tf-idf.csv", sep=";", decimal=",")

In [ ]:
# Wide-Format → Long-Format
konsum_long = konsum.melt(
    id_vars="Year",            # Zeitvariable behalten
    var_name="Stadt",          # Städtenamen aus Spalten
    value_name="Konsum_mg_tag" # Messwert: Drogenkonsum
)

# Einheitliche Jahresbezeichnung
konsum_long = konsum_long.rename(columns={"Year": "Jahr"})

# Fehlende Konsumwerte entfernen
konsum_long = konsum_long.dropna()

# Städtenamen vereinheitlichen (Merge-Kompatibilität)
konsum_long["Stadt"] = konsum_long["Stadt"].replace({
    "Munich G": "München",
    "Nuremberg": "Nuernberg",
    "Saarbrücken (2)": "Saarbruecken"
})

# Relevante Variablen aus Berichte-Datensatz filtern
berichte_sub = berichte[["Stadt", "Jahr", "Frequenz_Prozent"]].dropna()

# Zusammenführen beider Datensätze (Stadt × Jahr)
df = pd.merge(
    konsum_long,
    berichte_sub,
    on=["Stadt", "Jahr"],
    how="inner"           # nur vollständige Beobachtungen
)

# Spearman-Korrelation
rho, p = spearmanr(
    df["Konsum_mg_tag"],
    df["Frequenz_Prozent"]
)

# Testergebnisse ausgeben
print(f"Spearman rho = {rho:.3f}")
print(f"p-Wert = {p:.4f}")

# Hypothesenentscheidung mit α = 0.05
if p <= 0.05:
    print("➡ Signifikanter Zusammenhang")
else:
    print("➡ Kein signifikanter Zusammenhang")


Spearman rho = 0.032
p-Wert = 0.8411
➡ Kein signifikanter Zusammenhang


---
Ergebnis <br>
-> H0 wird verworfen <br>
-> H1 wird angenommen